In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

In [ ]:
dataset_path = '/mnt/d/Career/nano courses/opencv/done_data/gray/chess_color_dataset/'
train_path = dataset_path + 'train/'
test_path = dataset_path + 'test/'

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% for validation
)

train_gen = datagen.flow_from_directory(
    train_path,
    target_size=(44, 44),
    color_mode="grayscale",
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_path,
    target_size=(44, 44),
    color_mode="grayscale",
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
def count_images_per_class(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        print(class_name)
        class_dir = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_dir):
            continue  # Skip if not a directory

        num_images = len([f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        class_counts[class_name] = num_images

    return class_counts

print("\n✅ Number of images per class AFTER augmentation:")
after_counts = count_images_per_class(train_path)
for class_name, count in after_counts.items():
    print(f"  {class_name}: {count} images")

In [ ]:
def create_model():
    model = Sequential([
        Input(shape=(44, 44, 1)),  # Define input shape explicitly
        Conv2D(32, (3, 3), activation="relu"),  
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),  # Reduce overfitting
        Dense(1, activation="sigmoid")  # Binary output: Black (0) or White (1)
    ])
    
    # Compile the model
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Create the model
model = create_model()

# Print the model summary
model.summary()

In [ ]:
history = model.fit(train_gen, epochs=7, validation_data=val_gen)
# Save the trained model


In [ ]:
test_gen = datagen.flow_from_directory(
    test_path,
    target_size=(44, 44),
    color_mode="grayscale",
    batch_size=32,
    class_mode='binary'
)
model.evaluate(test_gen)

In [ ]:
model.save("./chess_color_classifier.h5")
# Save the trained model